In [ ]:
import torchvision,torch,time
from torchvision import datasets, transforms
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from vit_pytorch import ViT

# 设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))
import os
current_directory = os.getcwd()
print("Current working directory:", current_directory)


In [ ]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(64),       # 随机裁剪并调整大小
    transforms.RandomHorizontalFlip(),       # 随机水平翻转
    transforms.ToTensor(),                   # 转换为张量
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])
data_dir = '../dataset/Summer_camp/dataset1/'
train_data = datasets.ImageFolder(root=data_dir + 'train', transform=transform)
test_data = datasets.ImageFolder(root=data_dir + 'val', transform=transform)
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))
# 打印类别到索引的映射
print("Class to index mapping:", train_data.class_to_idx)

In [ ]:
    # img show test
    idx = 1256
    img, label = train_data[idx]
    print(img.shape)
    img = transforms.ToPILImage()(img)
    img.show()
    print("Label:", label)

In [ ]:
# 创建网络模型    
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.model1 = Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 9)
        )

    def forward(self, x):
        x = self.model1(x)
        return x
    

tudui = ViT(
    image_size=64,
    patch_size=8,
    num_classes=9,
    dim=256,
    depth=6,
    heads=16,
    mlp_dim=2048,
    dropout=0.1,
    emb_dropout=0.1,
).to(device)
loss_fn = nn.CrossEntropyLoss() # 交叉熵损失函数
loss_fn = loss_fn.to(device)
learning_rate = 1e-3
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)
input = torch.randn(64, 3, 64, 64).to(device)
output = tudui(input)
print(output.shape)

print()

In [ ]:
total_train_step = 0    # 训练次数
total_test_step = 0     # 测试次数
epoch = 50              # 训练轮数
writer = SummaryWriter("../logs/defect_raw")
last_time = time.time()
for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i+1))
    # 训练
    tudui.train() # 对某些特定的层需要此句
    for data in train_dataloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = tudui(inputs)
        loss = loss_fn(outputs, labels)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        # if total_train_step % 100 == 0:
        #     print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
        #     writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试
    tudui.eval() # 对某些特定的层需要此句
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = tudui(inputs)
            loss = loss_fn(outputs, labels)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == labels).sum() # argmax取最大值，argmax(0)竖着看，argmax(1)横着看
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    # torch.save(tudui, "../models/tudui_{}.pth".format(i))
    torch.save(tudui.state_dict(), "../models/defect_raw/tudui_{}.pth".format(i))
    delta_time = time.time() - last_time
    last_time = time.time()
    print("模型已保存，耗时：{}".format(delta_time))

writer.close()